# **Multigrams: Full Workflow**
The multigram workflow mirrors the unigram workflow, but with two differences. First, instead of _creating_ a whitelist, you filter the multigram corpus _using_ a whitelist containing the top-_N_ unigrams. Second, the multigram workflow adds a **pivoting** step. Pivoting reorganizes the database so that it's easy to query year-ngram combinations. For instance, you can learn how many times the word "nuclear" appeared in 2011 by querying the key `[2011] nuclear`. This is useful for analyzing changes in word meanings over time.

## **Setup**
### Imports

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from stop_words import get_stop_words
from ngramkit.ngram_filter.lemmatizer import SpacyLemmatizer
from ngramkit.ngram_acquire import download_and_ingest_to_rocksdb
from ngramkit.ngram_filter.config import PipelineConfig as FilterPipelineConfig
from ngramkit.ngram_filter.config import FilterConfig
from ngramkit.ngram_filter.pipeline.orchestrator import build_processed_db
from ngramkit.ngram_pivot.config import PipelineConfig as PivotPipelineConfig
from ngramkit.ngram_pivot.pipeline import run_pivot_pipeline
from ngramkit.utilities.peek import db_head, db_peek, db_peek_prefix

### Configure

In [2]:
db_path_stub = '/scratch/edk202/NLP_corpora/Google_Books/'
archive_path_stub = '/scratch/edk202/NLP_archive/Google_Books/'
release = '20200217'
language = 'eng'
size = 5

## **Step 1: Download and Ingest**

In [3]:
download_and_ingest_to_rocksdb(
    ngram_size=size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    archive_path_stub=archive_path_stub,
    ngram_type="tagged",
    random_seed=67,
    overwrite_db=True,
    workers=60,
    write_batch_size=5_000,
    open_type="write:packed24",
    compact_after_ingest=True
)

N-GRAM ACQUISITION PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-15 02:21:08

Download Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Ngram repo:           https://books.storage.googleapis.com/?prefix=ngrams/books/20200217/eng/5-
DB path:              /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
File range:           0 to 19422
Total files:          19423
Files to get:         19423
Skipping:             0
Download workers:     60
Batch size:           5,000
Ngram size:           5
Ngram type:           tagged
Overwrite DB:         True
DB Profile:           write:packed24

Download Progress
════════════════════════════════════════════════════════════════════════════════════════════════════


Files Processed: 100%|█████████████████████████████████████████████████| 19423/19423 [3:31:36<00:00]



Post-Ingestion Compaction
════════════════════════════════════════════════════════════════════════════════════════════════════
Initial DB size:         2.21 TB


Compaction failed: IO error: While pwrite to file at offset 104857600: /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db/003674.sst: Disk quota exceeded


Compaction failed: IO error: While pwrite to file at offset 104857600: /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db/003674.sst: Disk quota exceeded
Database is still usable, but may not be optimally compacted.

Database Archival
════════════════════════════════════════════════════════════════════════════════════════════════════
Compressing database to temporary location: ...1/job-445643/ngram_archive_djdaa7_i/5grams.db.tar.zst
Compressing /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db to /state/partition1/job-445643/ngram_archive_djdaa7_i/5grams.db.tar.zst
Using 128 threads at compression level 3
Compression complete: /state/partition1/job-445643/ngram_archive_djdaa7_i/5grams.db.tar.zst
Transferring archive to ...dk202/NLP_archive/Google_Books/20200217/eng/5gram_files/5grams.db.tar.zst
Archive created: /scratch/edk202/NLP_archive/Google_Books/20200217/eng/5gram_files/5grams.db.tar.zst

Processing complete!

Final Summary
═════

## **Step 2: Filter and Normalize**
`config.py` contains generic defaults for the filtering pipeline. You can override these defaults by passing option dictionaries to the `build_processed_db` function, as seen below. As implemented here, we use the whitelist from the unigram workflow to filter the multigram corpus. If we weren't using a whitelist, we could normalize, filter, and lemmatize each token just as we did for the unigrams.

In [4]:
setup_notebook_logging(
    workflow_name="multigrams_filter",
    data_path=str(base_path),
    console=False
)

stop_set = set(get_stop_words("english"))
lemmatizer = SpacyLemmatizer(language="en")

filter_config = FilterConfig(
    stop_set=stop_set,
    lemma_gen=lemmatizer,
    whitelist_path=whitelist_path
)

pipeline_config = FilterPipelineConfig(
    src_db=raw_db,
    dst_db=filtered_db,
    tmp_dir=filter_tmp_dir,
    num_workers=58,
    use_smart_partitioning=True,
    num_initial_work_units=1000,
    cache_partitions=True,
    use_cached_partitions=True,
    samples_per_worker=500_000,
    work_unit_claim_order="random",
    flush_interval_s=5.0,
    mode="resume",
    progress_every_s=15.0,
    ingest_num_readers=10,
    ingest_batch_items=1_000_000,
    ingest_queue_size=3,
    compact_after_ingest=True
)

build_processed_db(pipeline_config, filter_config)

N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Pipeline
Run mode:             resume
Compact after ingest: True

Workers
Num Workers:        58
Initial work units: 1000
Profiles:           read=read:packed24, write=write:packed24
Flush interval:     5.0s

Files
Source: /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
Destination: /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Input whitelist: ..._corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db/whitelist.txt
  All tokens (min count: 1)
Output whitelist: None
Loading whitelist...
Loaded 6,000 tokens

Phase 1: Creating work units...
════════════════════════════════════════════════════════════════════════════════════════════════════
Resuming existing work units
Resumi

## **Step 3: Pivot to Yearly Indices**

In [3]:
setup_notebook_logging(
    workflow_name="multigrams_pivot",
    data_path=str(base_path),
    console=False
)

pivot_config = PivotPipelineConfig(
    src_db=filtered_db,
    dst_db=pivoted_db,
    tmp_dir=pivot_tmp_dir,
    num_workers=50,
    num_initial_work_units=1000,
    cache_partitions=True,
    use_cached_partitions=True,
    samples_per_worker=500_000,
    work_unit_claim_order="random",
    flush_interval_s=15.0,
    progress_every_s=60.0,
    mode="reprocess",
    num_ingest_readers=1,
    ingest_buffer_shards=1,
    use_smart_partitioning=True,
    ingest_mode="direct_sst"
)

run_pivot_pipeline(pivot_config)


PARALLEL N-GRAM DATABASE PIVOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-08 22:08:32
Mode:       REPROCESS

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Target DB:            .../edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db
Temp directory:       /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/pivot_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              50
Initial work units:   1000

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       write:packed24
Ingest profile:       write:packed24



SST Files Ingested: 100%|██████████████████████████████████████████████████| 1000/1000 [01:54<00:00]



Phase 4: Finalizing database...
════════════════════════════════════════════════════════════════════════════════════════════════════

Post-Ingestion Compaction
────────────────────────────────────────────────────────────────────────────────────────────────────
Initial DB size:         259.69 GB
Compaction completed in 1:01:20
Size before:             259.69 GB
Size after:              631.86 GB
Space saved:             -372.17 GB (-143.3%)

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ PROCESSING COMPLETE                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Items: 18,092,372,962 (estimated)                                                                │
│ Size: 631.86 GB                                                                                  │
│ Database: /scratch/edk202/NLP_corpora/Google_Bo

# Inspect Final Database
Here are three functions you can use to inspect the final database.

## `db_head`: First N records

In [8]:
db_head(str(pivoted_db), n=100)

First 100 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [1470] <UNK> <UNK> <UNK> <UNK> eng
     Value: 1 occurrences in 1 documents

[ 2] Key:   [1470] <UNK> <UNK> <UNK> atomic energy
     Value: 1 occurrences in 1 documents

[ 3] Key:   [1470] <UNK> <UNK> <UNK> eng <UNK>
     Value: 1 occurrences in 1 documents

[ 4] Key:   [1470] <UNK> <UNK> <UNK> much convenient
     Value: 1 occurrences in 1 documents

[ 5] Key:   [1470] <UNK> <UNK> <UNK> selection <UNK>
     Value: 1 occurrences in 1 documents

[ 6] Key:   [1470] <UNK> <UNK> atomic energy commission
     Value: 1 occurrences in 1 documents

[ 7] Key:   [1470] <UNK> <UNK> commission <UNK> <UNK>
     Value: 3 occurrences in 2 documents

[ 8] Key:   [1470] <UNK> <UNK> desire <UNK> <UNK>
     Value: 1 occurrences in 1 documents

[ 9] Key:   [1470] <UNK> <UNK> employee <UNK> <UNK>
     Value: 2 occurrences in 1 documents

[10] Key:   [1470] <UNK> <UNK> 

## `db_peek`: Records starting from a key

In [50]:
db_peek(str(pivoted_db), start_key="[2002] attack <UNK> <UNK> world trade", n=5)

5 key-value pairs starting from 000007d261747461636b203c554e4b3e203c554e4b3e20776f726c64207472616465:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2002] attack <UNK> <UNK> world trade
     Value: 16,813 occurrences in 8,089 documents

[ 2] Key:   [2002] attack <UNK> <UNK> world war
     Value: 12 occurrences in 12 documents

[ 3] Key:   [2002] attack <UNK> <UNK> worm <UNK>
     Value: 9 occurrences in 9 documents

[ 4] Key:   [2002] attack <UNK> <UNK> worship <UNK>
     Value: 10 occurrences in 10 documents

[ 5] Key:   [2002] attack <UNK> <UNK> worth <UNK>
     Value: 3 occurrences in 3 documents



## `db_peek_prefix`: Records matching a prefix

In [31]:
db_peek_prefix(str(pivoted_db), prefix="[2011] poor <UNK> happy", n=5)

5 key-value pairs with prefix 000007db706f6f72203c554e4b3e206861707079:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2011] poor <UNK> happy <UNK> <UNK>
     Value: 92 occurrences in 79 documents

[ 2] Key:   [2011] poor <UNK> happy <UNK> rich
     Value: 11 occurrences in 11 documents

[ 3] Key:   [2011] poor <UNK> happy <UNK> sad
     Value: 20 occurrences in 19 documents

[ 4] Key:   [2011] poor <UNK> happy <UNK> unhappy
     Value: 13 occurrences in 13 documents

[ 5] Key:   [2011] poor <UNK> happy childhood <UNK>
     Value: 7 occurrences in 7 documents

